# Machine Learning for EEG Dataset : DEAP

http://www.eecs.qmul.ac.uk/mmv/datasets/deap/

<h1> Extracting Feature Vector as follows and saving to file </h1>

<b> Features  - {SE1, SE2, O, Stat1,...., Stat8, PSD, LogPSD, Hemi, HdSE1, HdSE2, HdO, HFD}</b>

- SE1 - Shannon Entropy 1 - Traditional method
- SE2 - Shannon Entropy 2 - Spectral Entropy for Time series
- O - Oscillation Feature
- Stat1-8 : 8 Statistical Features
    - Mean
    - Standard Deviation
    - Skewness
    - Kurtosis
    - Mean of First Difference of Raw Signal
    - Mean of First Difference of Normalized Signal
    - Mean of Second Difference of Raw Signal
    - Mean of Second Difference of Normalized Signal
- PSD - Absolute power of the Power Spectral Density Spectrum
- LogPSD - Logarithm of Absolute power of the Power Spectral Density Spectrum
- Hemi - Hemispheric mean difference of raw signals 
- HdSE1, HdSE2, HdO - Hemispheric difference between the Shannon entropy and oscillation features
- HFD - Higuchi Fractal Dimension Feature

nFeatures = 18 for each frequency band <br>
nFeatures for each trial = 32 (electrode channels) x 5(Freq bands) x 18 features
                         <br> <b>=  2880 features per trial per patient
 

_______________

## 1. Imports

In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import butter,filtfilt, welch
import scipy.signal as sg
from scipy.fftpack import fft, ifft
import scipy.stats as sts
from scipy.integrate import simps
import glob
import os
import ctypes
import numpy as np
from numpy.ctypeslib import ndpointer

________________

## 2. Pre-processing Functions

#### 2.1 Function to Initialize the Global Variables across all files

In [2]:
#Initialize global variables like time, channel names and indices and cut-off frequencies for filters
def global_run():
    time = np.linspace(0,8063/128,8064)
    channel_names = ["Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7","CP5", "CP1", "P3", "P7", "PO3", "O1", "Oz", "Pz", 
                "Fp2", "AF4", "Fz", "F4", "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8", "PO4", "O2"]
    channel_labels = dict(list(enumerate(channel_names)))
    cutoffs = np.array([[0.5, 4], [4, 7.5], [7.5, 12], [12, 30], [30, 75]])
    ind_dict = hemi_run(channel_labels)
    return time, channel_labels, cutoffs, ind_dict


In [3]:
#Function to form a relation between the indexes of left and right electrodes.
def hemi_run(channel_labels):
    
    hemi = {
        "Fp1":"Fp2", "AF3":"AF4", "F3":"F4", "F7":"F8", "FC5":"FC6", "FC1":"FC2", "C3":"C4", "T7":"T8",
        "CP5":"CP6", "CP1":"CP2", "P3":"P4", "P7":"P8", "PO3":"PO4","O1":"O2","Fp2":"Fp1", "AF4":"AF3", 
        "F4":"F3", "F8":"F7", "FC6":"FC5", "FC2":"FC1", "C4":"C3", "T8":"T7","CP6":"CP5", "CP2":"CP1", 
        "P4":"P3", "P8":"P7", "PO4":"PO3","O2":"O1"}
    
    channel_ind = {v: k for k, v in channel_labels.items()}
    
    ind_dict = {}
    for i in range(32):
        lab = channel_labels[i]
        opp = [hemi[lab] if lab in hemi.keys() else False][0]
        if not opp:
            ind_dict[i]=np.nan
        else:
            ind = channel_ind[opp]
            ind_dict[i] = ind

    return ind_dict

#### 2.2 Load Data

In [4]:
#Load data from file path
def load_data(file):
    with open(file, 'rb') as f:
        loaded = pickle.load(f, encoding="bytes")  #the specification of encoding is to supress errors in loading the file.
    x = loaded[b'data']
    data = x[:,:32, :] #Choosing only the 32 EEG Channels
    if data.shape != (40,32,8064):
        print('Error. Shape mismatch')
        return None
    labels = loaded[b'labels']
    return data, labels

#### 2.3 Tranforming the Labels based on 1D-2CLS Labelling Scheme

In [5]:
#Convert the Labelling Scheme into 1D-2CLS scheme and remove Liking Label
def label_scheme(labels):
    return ((labels>5)*1)[:,:3]

#### 2.4 FIR Filter functions

In [6]:
#FIR filter to split the signal into the frequency bands
def FIR_filter(data, tim, fs = 128, n = 51):

    #Filter the data based on the different cut-off frequencies for each MEG band
    cutoffs = np.array([[0.5, 4], [4, 7.5], [7.5, 12], [12, 30], [30, 75]]) #Cutoff frequencies for the bands - delta, theta, alpha, beta gamma respectively
    if fs/2<75: #cutoff frequency cannot exceed fs/2
        cutoffs[4,1] = 128/2 - 1
    
    splits = []
    for i in range(5):
        #Desigining the FIR filter with the specific cut-off frequency and window to get the filter co-efficients
        hfilt = sg.firwin(n, cutoff = cutoffs[i]/fs, window = 'blackmanharris', pass_zero = False)
        #Convolve the signal with filter to get filered signal
        yfilt = sg.convolve(data,hfilt, mode="same")
        splits.append(yfilt)
    
    return np.array(splits) #Delta Theta Alpha Beta Gamma

In [7]:
def get_filtered_data(data, tim, fs=128, n=51):
    filtered = np.apply_along_axis(FIR_filter, 2, data, time, n=n)
    if filtered.shape != (40, 32, 5, 8064):
        print("Error: Shape Mismatch")
        return None
    return filtered
    

________

## 3.Feature Functions

#### 3.1 Shannon Entropy

In [8]:
#Traditional Shannon Entropy
def shannon_entropy(x):
    weights = np.ones_like(x) / len(x)
    px = np.histogram(x, weights=weights)[0]
    px = px[px!=0]
    return -sum(px*np.log(px))

In [9]:
#Shannon entropy based on Spectral Features
def spectral_entropy(x, fs, nperseg=None, normalize=False):
    x = np.array(x)
    # Compute and normalize power spectrum
    _, psd = welch(x, fs, nperseg=nperseg)
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    if normalize:
        se /= np.log2(psd_norm.size)
    return se

#### 3.2 PSD

In [10]:
#Absolute power and Logarithmic Power of PSD of the Signal with specified Frequency Band
def PSD_power(data, fs, freq, nparseg=1024):
    f, Pxx = welch(data, fs, nperseg=nparseg)
    freq_res = f[1] - f[0]
    idx = np.logical_and(f >= freq[0], f <= freq[1])
    power = simps(Pxx[idx], dx=freq_res)
    return power, np.log(power)

In [11]:
def get_PSDFeats(sigs, cutoffs, fs=128):
    if len(sigs)!=len(cutoffs):
        print("Error: Wrong input,Length mismatch")
        return None
    PSD = []
    for i in range(len(sigs)):
        PSD.append(PSD_power(sigs[i], fs, cutoffs[i]))
    PSD = np.array(PSD)
    if PSD.shape!=(5,2):
        print("Error: Shape mismatch")
        return None
    return PSD

#### 3.3 Oscillation

In [12]:
#Oscillation Feature
def oscillation_signal(x):
    N = len(x)
    O = N/(len(sg.find_peaks(x)[0]) + len(sg.find_peaks(-x)[0]))
    return O

#### 3.4 Statistical Features

In [13]:
#Normalising function of a single signal
def norm(x):
    x = (x-min(x))/(max(x)-min(x))
    return x

In [14]:
#Calculating 8 Statistical Features of Signal Input
def stats(x, s=0,h=0):
    mean = np.mean(x)
    std = np.std(x)
    skew = sts.skew(x)
    kurt = sts.kurtosis(x)
    norm_x = norm(x)
    #np.diff calculates the first difference of input array
    fd_raw = np.mean(abs(np.diff(x))) 
    fd_norm = np.mean(abs(np.diff(norm_x)))
    sd_raw = np.mean(abs(np.diff(np.diff(x))))
    sd_norm = np.mean(abs(np.diff(np.diff(norm_x))))
    return np.array([mean, std, skew, kurt, fd_raw, fd_norm, sd_raw, sd_norm])
    

#### 3.5 HFD

<b><u> Note:</b>
- I transferred the HFD function from the python file to the notebook itself.
- You can still use the one from the file as well, I just thought this might speed up the process

In [15]:
#Functions for Calculating Higuchi Fractal Dimension (HFD)
def hfd(X,**kwargs):
    k, L = curve_length(X,**kwargs)
    return lin_fit_hfd(k, L);

def curve_length(X,opt=False,num_k=50):
    ### Make sure X is a NumPy array with the correct dimension
    X = np.array(X)
    if X.ndim != 1:
        raise ValueError("Input array must be 1D (time series).")
    N = X.size

    ### Get interval "time"
    k_arr = interval_t(N,num_val=num_k)

    ### The average length
    Lk = np.empty(k_arr.size,dtype=np.float)

    for i in range(k_arr.size):# over array of k's
        Lmk = 0.0
        for j in range(k_arr[i]):# over m's
            ## Construct X_k^m, i.e. X_(k_arr[i])^j, as X[j::k_arr[i]]
            ## Calculate L_m(k)
            Lmk += (np.sum(np.abs(np.diff( X[j::k_arr[i]] ))) * (N - 1) /(( (N-j-1)//k_arr[i] )*k_arr[i])) / k_arr[i]

        ### Calculate the average Lmk
        Lk[i] = Lmk / k_arr[i]

    return (k_arr, Lk);

def lin_fit_hfd(k,L,log=True):
    return (-np.polyfit(np.log2(k),np.log2(L),deg=1)[0]);
    
def interval_t(size,num_val=50):
    ### Generate sequence of interval times, k
    k_stop = size//2
    if k_stop > size//2:## prohibit going larger than N/2
        k_stop = size//2
        print("Warning: k cannot be longer than N/2")
        
    k = np.logspace(start=np.log2(2),stop=np.log2(k_stop),base=2,num=num_val,dtype=np.int)
    return np.unique(k);


Note: 
- PSD features and hemispheric features are calculated together (in the following section), as they require the same iter values during calculation

_____

## 4. Feature Extraction functions

#### 4.1 Calculating the first 11 features separately - feature_vector()

In [16]:
def feature_vector(x, fs=128):
    '''
    Returns a Feature Vector in the following order:
    [SE1, SE2, O, Stat1...Stat8, PSD, LogPSD]
    
    SE1: Shannon Entropy (Traditional)
    SE2: Spectral Entropy 
    O: Oscillation Feature
    StatsX: 8 statistical features (refer stats function)
    
    '''
    feat_vec = []
    
    feat_vec.append(shannon_entropy(x))
    feat_vec.append(spectral_entropy(x,fs))
    feat_vec.append(oscillation_signal(x))
    feat_vec += list(stats(x))
    if len(feat_vec)!=11:
        print("Error: Shape Mismatch")
        return None
    return np.array(feat_vec)
    

#### 4.2 Calculating the hemispheric and PSD features - hemi_diffs()

In [17]:
#Calculating differences in hemispheric features 
def hemi_diffs(sig, feat, cutoffs, ind_dict):
    features = feat.copy()
    channel_feats = []
    for ch in range(len(sig)):
        opp = ind_dict[ch]
        if np.isnan(opp):
            #Check for nan values - centre electrodes
            #If centre ('z') electrode-  append nan values as features
            fleft = features[ch]
            feat_diff = np.concatenate((fleft, np.zeros((5,6))*np.nan), axis=1)
        else:
            #For left/right electrodes
            #Extract the corresponding signal and features from the opposite side
            left = sig[ch] #Shape: (5,8064)
            right = sig[opp] #Shape: (5,8064)
            mdiff = np.mean(left-right, axis=1) #Shape: (5,)
            
            #Use the iter value to calculate the PSD features as well
            PSD = get_PSDFeats(left, cutoffs) #Shape: (5,2)
            
            #Get the calculated features - Oscillation and Shannon Entropy to find difference between left/right hemispeheres
            fleft = features[ch] #Shape: (5,11)
            fright = features[opp] #Shape: (5,11)
            f_diff = fleft[:,[0,1,2]] - fright[:,[0,1,2]] #Shape: (5,3)
            
            #Concatenate all the feature values to get the feature vector of shape (5,17)
            feat_diff = np.concatenate((fleft,PSD, mdiff.reshape(5,1),f_diff), axis=1)
        
        #Sanity check
        if feat_diff.shape != (5,17):
            print("Error:Shape mismatch")
            return None
        channel_feats.append(feat_diff)
    return np.array(channel_feats)        

#### 4.3 Concatenate all calculated features across the electrodes - get_features()

In [18]:
#This function is a global version of the previous hemi_diffs() function
#This calculates the feature vector for each trial 
#This function is used for each .dat file

def get_features(filtered, cutoffs, ind_dict):
    features = []
    #Calculate the first 12 features using numpy's vectorization to apply it along the entire dataset
    first_feats = np.apply_along_axis(feature_vector, 3, filtered)
    
    #Then iterate over the trials to calculate the electrode features for each trial
    for i in range(len(filtered)):
        feats = hemi_diffs(filtered[i], first_feats[i], cutoffs, ind_dict)
        
        #Sanity Check
        if feats.shape != (32,5,17):
            print("Error:Shape mismatch")
            return None
        
        features.append(feats)
    #Final feature array
    return np.array(features)

_____

## 5. Final function to Get all Features and Save the features to files

- Combined function that includes all previous function to be executed in a single line
- Iterates through all the files in the specified directory ("path"), extracts all the features 
- and saves it to a pickle file of the same name

In [19]:
#Combined function that includes all previous function to be executed in a single line
#Iterates through all the files in the specified directory ("path"), extracts all the features 
# and saves it to a pickle file of the same name
def all_features(path):
    '''
    Input path must be the directory that holds all the .dat files of the dataset.
    
    Output files are saved in this same directory
    '''
    #Iter over each file in the directory using glob
    for filename in glob.glob(path+'\s[0-9]**.dat'):
        #load the data
        data, labels = load_data(filename)
        
        #Pre-processing - filter the data
        filtered = get_filtered_data(data, time)
        
        #Extract the features of the filtered data
        features = get_features(filtered, cutoffs, ind_dict)
        
        #Santiy Check
        if features.shape != (40,32,5,17):
            print("Error:Shape mismatch")
            print(filename[:-7])
            break
        
        #Create output dictionary to be saved
        feat_dict = {'features':features,
                    'labels':label_scheme(labels)} #Convert the labels into the 1D-2CLS labelling scheme
        
        #Print to keep track of progress
        print("File " + filename[-6:-4] + " : done")
        
        #Save file with same name but .pickle extension
        #Files are saved in the same directory as the input files
        with open(filename[:-4]+'.pickle', 'wb') as handle:
            pickle.dump(feat_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

________

## 6. Execution: Main Code

In [2]:
#Change this path to include the directory (folder) that holds all the .dat files 
data_path =  r"C:\Users\Documents\Machine Learning\Upwork Project\data_preprocessed_python\data_preprocessed_python"

In [6]:
#Initialization of global variables
time, channel_labels, cutoffs, ind_dict = global_run()

In [ ]:
load_data()

In [ ]:
#Run this to extract all features and save the extracted features into pickled files
all_features(data_path)

#### Check if the data is saved correctly

In [10]:
data_path = r"C:\Users\Documents\Machine Learning\Upwork Project\data_preprocessed_python\data_preprocessed_python\Final_features\feats07.pickle"

In [11]:
#Chosing a random file
with open(data_path, 'rb') as handle:
    pickled = pickle.load(handle)
    
#loaded = pickle.load(open('s01.dat', 'rb'), encoding="bytes")

In [12]:
pickled.keys()

dict_keys(['features', 'labels'])

In [13]:
pickled['features'].shape

(40, 32, 5, 18)

In [14]:
pickled['labels'].shape

(40, 3)

In [15]:
Features = pickled['features']

__________

In [16]:
Features.shape

(40, 32, 5, 18)

In [17]:
Features

array([[[[ 1.15360416e+00,  2.44024820e+00,  1.27393365e+01, ...,
          -1.17594423e-02, -1.83740430e-01,  1.94387958e+00],
         [ 1.07276590e+00,  2.91658903e+00,  1.08532974e+01, ...,
           7.76333035e-02, -3.77900329e-01,  1.95519661e+00],
         [ 1.06389469e+00,  3.42353344e+00,  9.16363636e+00, ...,
           1.12400423e-01, -2.13107822e-01,  1.97049993e+00],
         [ 1.23734342e+00,  4.17840877e+00,  6.01791045e+00, ...,
          -8.69754946e-02,  5.34134063e-02,  1.99397285e+00],
         [ 1.49619776e+00,  5.25959171e+00,  2.56733524e+00, ...,
           2.14236670e-03, -1.14943055e-02,  2.00568438e+00]],

        [[ 1.37966411e+00,  2.62996766e+00,  1.26197183e+01, ...,
           9.92223386e-02,  1.97183099e-02,  1.96090281e+00],
         [ 1.38825071e+00,  3.18018521e+00,  1.08532974e+01, ...,
           1.88876815e-01, -1.63096000e-01,  1.97019683e+00],
         [ 1.36344071e+00,  3.52016728e+00,  8.86153846e+00, ...,
           6.82510364e-02, -3.439409

In [19]:
pickled['labels']

array([[1, 0, 0],
       [1, 1, 0],
       [1, 0, 0],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 0, 1],
       [1, 1, 1],
       [1, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 1],
       [1, 1, 0],
       [0, 1, 1],
       [0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [0, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])